In [ ]:
def process_feedback_with_ai(feedback_data, modify_specific_session=False):
    try:
        # Fetch the current workout plan
        current_plan = feedback_data['workout_plan']
        if modify_specific_session:
            # Extract the specific session from the workout plan
            session_name = feedback_data['session_name']
            current_session = None
            for day in current_plan.get('workoutDays', []):
                if day['day'] == session_name:
                    current_session = day
                    break
            if not current_session:
                logger.error(f"Session '{session_name}' not found in the current workout plan.")
                return None
            current_session_json = json.dumps(current_session, indent=2)
            prompt = f"""
            The user {feedback_data['user']['username']} has provided feedback for the session '{feedback_data['session_name']}'.
            They rated it as '{feedback_data['emoji_feedback']}' and commented: '{feedback_data['comments']}'.
            Below is the current workout plan for '{feedback_data['session_name']}':
            ```json
            {current_session_json}
            ```
            Please analyze the current session and make necessary adjustments based on the user's feedback.
            Important Instructions:
            Do not change the session day name.
            Only modify the exercises, sets, reps, or instructions for the specified session '{session_name}'.
            Do not include any other sessions in your response.
            Provide the modified session in JSON format as per the schema:
            Please analyze the current workout plan and make necessary adjustments to the session '{feedback_data['session_name']}' based on the user's feedback.
            Do not change or shuffle the session days.
            Only modify the exercises, sets, reps, or instructions for the specified session '{feedback_data['session_name']}'.
            Keep the rest of the workout plan unchanged.
            Provide the modified session in JSON format as per the schema:
            ```json
            {{
                "workoutDays": [
                    {{
                        "day": "{feedback_data['session_name']}",
                        "duration": "60 minutes",
                        "exercises": [
                            {{
                                "name": "Exercise Name",
                                "setsReps": "Sets and Reps",
                                "equipment": "Equipment",
                                "instructions": "Instructions"
                            }},
                            // ... more exercises ...
                        ]
                    }}
                ]
            }}
            ```
            **Instructions:**
            1. **Provide only the JSON response without any additional explanations or text.**
            2. **Ensure the JSON is properly formatted and adheres strictly to the provided schema.**
            3. **Do not include any text before or after the JSON.**
            4. **If you need to mention anything, include it within the JSON as 'additionalTips'.**
            """
        else:
            prompt = f"""
            The user {feedback_data['user']['username']} has provided feedback on their workout plan.
            They rated it as '{feedback_data['emoji_feedback']}' and commented: '{feedback_data['comments']}'.
            Below is the current workout plan:
            ```json
            {current_plan_json}
            ```
            **Important Instructions:**
            Please analyze the current workout plan and make necessary adjustments based on the user's feedback.
            Do not change or shuffle the session days.
            Do not create a new workout plan based on the user's feedback.
            Keep the sequence of training days and exercises the same.
            Only modify the exercises, sets, reps, or instructions where needed.
            Provide the modified workout plan in JSON format as per the existing schema.
            **Instructions:**
            1. **Provide only the JSON response without any additional explanations or text.**
            2. **Ensure the JSON is properly formatted and adheres strictly to the provided schema.**
            3. **Do not include any text before or after the JSON.**
            4. **If you need to mention anything, include it within the JSON as 'additionalTips'.**
            """
        # Get the Replicate API token
        replicate_api_token = os.environ.get('REPLICATE_API_TOKEN')
        if not replicate_api_token:
            logger.error("Replicate API token not found in environment variables.")
            return None
        os.environ["REPLICATE_API_TOKEN"] = replicate_api_token  # Set the environment variable
        client = replicate.Client(api_token=replicate_api_token, timeout=300)
        # Define the model name
        model_name = "meta/meta-llama-3-70b-instruct"
        # Define the inputs
        inputs = {
            "prompt": prompt,
            "temperature": 0.7,
            "max_new_tokens": 3500,
            # Add other parameters if needed
        }
        # Run the model and collect the output
        output = client.run(
            model_name,
            input=inputs,
        )
        # Log the AI output
        logger.debug(f"AI Output String: {output}")
        # Convert output to string if it's a list
        if isinstance(output, list):
            output_str = ''.join(output)
        else:
            output_str = str(output)
        # Extract JSON from the output
        json_match = re.search(r'\{.*\}', output_str, re.DOTALL)
        if json_match:
            json_content = json_match.group(0)
        else:
            logger.error("No JSON content found in AI output")
            raise ValueError("No JSON content found in AI output")
        # Parse the JSON content
        try:
            workout_plan = json.loads(json_content)
        except json.JSONDecodeError as e:
            logger.error(f"JSON Decode Error: {e}")
            logger.error(f"Problematic AI Output: {json_content}")
            raise ValueError(f"Invalid JSON format after extraction: {e}")
        # Validate the JSON structure
        if validate_workout_plan(workout_plan):
            return workout_plan
        else:
            raise ValueError("Transformed workout plan JSON does not adhere to the required schema.")
    except replicate.exceptions.ReplicateError as e:
        logger.error(f"Replicate API Error: {e}", exc_info=True)
        raise ReplicateServiceUnavailable(f"Replicate API Error: {e}")
    except Exception as e:
        logger.error(f"Unexpected error generating workout plan: {e}", exc_info=True)
        raise

